# 第一章：基础提示结构

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [ ]:
# 安装OpenAI库
%pip install openai==1.61.0

# 导入Python内置的正则表达式库
import re
import openai

# 从IPython存储中检索API_KEY和MODEL_NAME变量
%store -r API_KEY
%store -r MODEL_NAME

# 如果没有设置MODEL_NAME，使用默认值
try:
    MODEL_NAME
except NameError:
    MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

# 创建OpenAI客户端
client = openai.OpenAI(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    """
    获取GPT的完成响应
    
    参数:
        prompt (str): 用户提示
        system_prompt (str): 系统提示（可选）
    
    返回:
        str: GPT的响应文本
    """
    # 构建消息列表
    messages = []
    
    # 如果有系统提示，添加系统消息
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    
    # 添加用户消息
    messages.append({"role": "user", "content": prompt})
    
    # 调用OpenAI API
    response = client.chat.completions.create(
        model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
        messages=messages,             # 消息列表
        max_tokens=2000,              # 最大token数
        temperature=0.0               # 温度参数，0表示更确定性
    )
    return response.choices[0].message.content

---

## 课程

OpenAI提供了强大的[Chat Completions API](https://platform.openai.com/docs/api-reference/chat)来与GPT模型进行交互。在本教程中，我们将使用Chat Completions API。

使用Chat Completions API调用GPT模型最少需要以下参数：
- `model`：您要使用的模型名称，如`gpt-4o`或`deepseek-r1`

- `max_tokens`：停止前要生成的最大token数。请注意，GPT可能在达到这个最大值之前就停止。此参数仅指定要生成的绝对最大token数。此外，这是一个*硬*停止，意味着它可能导致GPT在单词或句子中间停止生成。

- `messages`：输入消息数组。模型被训练为在交替的`user`和`assistant`对话轮次上操作。创建新对话时，您使用messages参数指定先前的对话轮次，然后模型生成对话中的下一个响应。
  - 每个输入消息必须是一个包含`role`和`content`的对象。您可以指定单个`user`角色消息，或者可以包含多个`user`和`assistant`消息（如果是这样，它们必须交替）。第一条消息可以是`system`消息（用于系统提示），然后是`user`消息。

还有可选参数，例如：
- `system`消息：系统提示 - 下面会详细介绍。
  
- `temperature`：GPT响应的变化程度。对于这些课程和练习，我们将`temperature`设置为0。

有关所有API参数的完整列表，请访问OpenAI的[API文档](https://platform.openai.com/docs/api-reference/chat)。

### 示例

让我们看看GPT如何响应一些格式正确的提示。对于以下每个单元格，运行单元格（`shift+enter`），GPT的响应将出现在代码块下方。

In [ ]:
# 提示
PROMPT = "Hi GPT, how are you?"

# 打印GPT的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = "Can you tell me the color of the ocean?"

# 打印GPT的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = "What year was Celine Dion born in?"

# 打印GPT的响应
print(get_completion(PROMPT))

现在让我们看看一些不包含正确Chat Completions API格式的提示。对于这些格式错误的提示，OpenAI API会返回错误。

首先，我们有一个Chat Completions API调用示例，在`messages`数组中缺少`role`和`content`字段。

In [ ]:
# 获取GPT的响应（错误格式示例）
try:
    response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi GPT, how are you?"}  # 缺少role和content字段
        ]
    )
    # 打印GPT的响应
    print(response.choices[0].message.content)
except Exception as e:
    print(f"API调用错误: {e}")
    print("错误原因: 消息缺少必需的'role'和'content'字段")

这是一个未能在`user`和`assistant`角色之间交替的提示示例。

In [ ]:
# 获取GPT的响应（错误格式示例）
try:
    response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}  # 两个连续的user消息，OpenAI允许这样做
        ]
    )
    # 打印GPT的响应
    print(response.choices[0].message.content)
except Exception as e:
    print(f"API调用错误: {e}")
    print("注意: OpenAI API实际上允许连续的user消息，这与Anthropic不同")

对于OpenAI的Chat Completions API，虽然`user`和`assistant`消息通常交替出现以获得最佳对话效果，但**API允许连续的user消息**。消息可以以`system`消息开始（用于系统提示），然后是`user`消息。您可以在提示中有多个`user`和`assistant`对（就像模拟多轮对话一样）。您还可以在assistant消息中放入部分响应，让GPT从您停下的地方继续（更多内容将在后面的章节中介绍）。

#### 系统提示

您还可以使用**系统提示**。系统提示是在"用户"轮次向GPT提出问题或任务之前**为GPT提供上下文、指令和指导方针**的一种方式。

在OpenAI API中，系统提示通过在消息列表开头添加一个`role`为`system`的消息来实现（查看notebook的[设置](#setup)部分中`get_completion`辅助函数的结构）。

在本教程中，无论我们在哪里可能使用系统提示，我们都在完成函数中为您提供了一个`system_prompt`参数。如果您不想使用系统提示，只需将`SYSTEM_PROMPT`变量设置为空字符串即可。

#### 系统提示示例

In [ ]:
# 系统提示
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# 用户提示
PROMPT = "Why is the sky blue?"

# 打印GPT的响应
print(get_completion(PROMPT, SYSTEM_PROMPT))

为什么要使用系统提示？**编写良好的系统提示可以在多个方面改善GPT的表现**，例如提高GPT遵循规则和指令的能力。有关更多信息，请访问OpenAI关于[系统消息最佳实践](https://platform.openai.com/docs/guides/prompt-engineering)的文档。

现在我们将深入一些练习。如果您想在不更改上述任何内容的情况下实验课程提示，请滚动到课程notebook的最底部访问[**示例演练场**](#example-playground)。

---

## 练习
- [练习 1.1 - 数到三](#exercise-11---counting-to-three)
- [练习 1.2 - 系统提示](#exercise-12---system-prompt)

### 练习 1.1 - 数到三
使用正确的`user` / `assistant`格式，编辑下面的`PROMPT`让Claude**数到三**。输出还将指示您的解决方案是否正确。

In [ ]:
# 提示 - 这是您应该更改的唯一字段
PROMPT = "[Replace this text]"

# 获取Claude的响应
response = get_completion(PROMPT)

# 评分练习正确性的函数
def grade_exercise(text):
    """检查文本是否包含数字1、2、3"""
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# 打印Claude的响应和相应的分数
print(response)
print("\n--------------------------- 评分 ---------------------------")
print("本练习已正确解决:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
# 导入并显示练习1.1的提示
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

### 练习 1.2 - 系统提示

修改`SYSTEM_PROMPT`，让Claude像3岁小孩一样回应。

In [ ]:
# 系统提示 - 这是您应该更改的唯一字段
SYSTEM_PROMPT = "[Replace this text]"

# 用户提示
PROMPT = "How big is the sky?"

# 获取Claude的响应
response = get_completion(PROMPT, SYSTEM_PROMPT)

# 评分练习正确性的函数
def grade_exercise(text):
    """检查响应是否包含像3岁小孩一样的表达方式"""
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# 打印Claude的响应和相应的分数
print(response)
print("\n--------------------------- 评分 ---------------------------")
print("本练习已正确解决:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
# 导入并显示练习1.2的提示
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

### 恭喜！

如果您已经解决了到此为止的所有练习，您就可以进入下一章了。愉快的提示工程！

---

## 示例演练场

这是一个让您自由实验本课程中显示的提示示例的区域，您可以调整提示来看看它如何影响Claude的响应。

In [ ]:
# 提示
PROMPT = "Hi Claude, how are you?"

# 打印Claude的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = "Can you tell me the color of the ocean?"

# 打印Claude的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = "What year was Celine Dion born in?"

# 打印Claude的响应
print(get_completion(PROMPT))

In [ ]:
# 获取Claude的响应（错误格式示例）
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}  # 缺少role和content字段
        ]
    )

# 打印Claude的响应
print(response[0].text)

In [ ]:
# 获取Claude的响应（错误格式示例）
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}  # 两个连续的user消息，违反了交替规则
        ]
    )

# 打印Claude的响应
print(response[0].text)

In [ ]:
# 系统提示
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# 用户提示
PROMPT = "Why is the sky blue?"

# 打印Claude的响应
print(get_completion(PROMPT, SYSTEM_PROMPT))